In [ ]:
# This code is using BERT sentiment analysis on Ethereum Magician comments
# we have broken comments into individualized comments by one person
# the code will first break each comment into 152 words chucks 
# run BERT Analysis
# aggregate based on average or mode of the sentiment predictor 
# this is a learning process 


In [1]:
import pandas as pd
import time

In [2]:
# get data 
data = pd.read_csv("C:/Users/moazz/Box/Fintech Research Lab/Ethereum Governance Project/magician_comments_separated.csv")

In [3]:
data['Discussion_Length'].describe()

count      434.000000
mean       865.062212
std       1132.073086
min          0.000000
25%         48.500000
50%        465.500000
75%       1297.250000
max      10194.000000
Name: Discussion_Length, dtype: float64

In [4]:
data.columns

Index(['EIP_Number', 'Title', 'Website', 'Author', 'Article_Body', 'Comments',
       'Comment_Authors', 'Discussion_Length'],
      dtype='object')

In [5]:
# create dataset that remove stopwords, puntuation, and lower cases
import string
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\moazz\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
# create a function for removing stop words, punctuation, and lowering the case
def preprocess(text):
    text = str(text)
    if text == 'nan':
        return ''          
    else :
        text = text.translate(str.maketrans('','',string.punctuation))
        text = text.lower()
        stop_words = set(stopwords.words('english'))
        words = text.split()
        mod_words = [word for word in words if word not in stop_words]
        mod_text = ' '.join(mod_words)
        return ''.join(mod_text) 

In [7]:
# apply the preprocess functin to comments
data['mod_comment'] = data['Comments'].apply(preprocess)

In [8]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\moazz\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [9]:
# create a function that create tokens for split senteces, these tokens will be included in list which
# breaks tokens into separate elements in a list to be further processed for BERT sentiment analysis
def tokenize_comment(comment):
    if pd.isnull(comment):
        return []
    elif len(str(comment)) > 512:
        tokenized_sentences = nltk.sent_tokenize(str(comment))
        if len(tokenized_sentences) == 0:
            return [str(comment)]
        else:
            return tokenized_sentences
    else:
        return [str(comment)]

In [10]:
data['Tokens'] = data['Comments'].apply(tokenize_comment)

In [11]:
# now I will run BERT Sentiment Analysis
from transformers import pipeline
sentiment_analysis = pipeline('sentiment-analysis', framework='pt', model='nlptown/bert-base-multilingual-uncased-sentiment')

In [15]:
data.loc[231,'EIP_Number']

3664

In [ ]:
# Apply Sentiment Analysis Model to selected cells and time it
start_time = time.time()
data['sentiment_scores'] = data['Tokens'].head().apply(lambda x: [sentiment_analysis(sentence)[0]['score'] for sentence in x if sentence])
#data['sentiment_scores'] = data['Tokens'].apply(lambda x: [sentiment_analysis(sentence)[0]['score'] for sentence in x if sentence and len(x) > 0])
end_time = time.time()
total_time = end_time - start_time
print(total_time)

In [12]:
sentiment_analysis(data.loc[39,'Tokens'])

[{'label': '2 stars', 'score': 0.4502643346786499},
 {'label': '2 stars', 'score': 0.37088510394096375},
 {'label': '3 stars', 'score': 0.3601614534854889},
 {'label': '3 stars', 'score': 0.4133625626564026},
 {'label': '3 stars', 'score': 0.5280786156654358},
 {'label': '3 stars', 'score': 0.32122278213500977},
 {'label': '3 stars', 'score': 0.2839789092540741},
 {'label': '2 stars', 'score': 0.4537961483001709},
 {'label': '5 stars', 'score': 0.4955727159976959},
 {'label': '5 stars', 'score': 0.3309934139251709},
 {'label': '4 stars', 'score': 0.35804206132888794},
 {'label': '3 stars', 'score': 0.5477418899536133},
 {'label': '3 stars', 'score': 0.5205487608909607},
 {'label': '4 stars', 'score': 0.2996140420436859},
 {'label': '4 stars', 'score': 0.47749024629592896},
 {'label': '3 stars', 'score': 0.26351380348205566}]

In [ ]:
# Apply Sentiment Analysis Model to all cells 
data['sentiment_scores'] = data['Tokens'].apply(lambda x: [sentiment_analysis(sentence)[0]['score'] for sentence in x if sentence])

In [24]:
# run a loop to record the processing time and error exception
data['score'] = [[]for _ in range(len(data))]
for i, token in enumerate(data['Tokens']):
    begin_time = time.time()
    row_score_list = []
    try:
        scores = sentiment_analysis(token)
        for score in scores:
            row_score_list.append({'label': score['label'], 'score': score['score']})
        data.loc[i,'score'].append(row_score_list)      
    except Exception as e:
        print(f"Error processing row {i}: {e}")
        continue
    end_time = time.time()
    processing_time = end_time - begin_time
    print(f"Time to Process {i}: {processing_time}")
        

Time to Process 0: 41.430556774139404
Time to Process 1: 3.725961923599243
Time to Process 2: 551.9060225486755
Time to Process 3: 39.49200391769409
Time to Process 4: 0.0
Time to Process 5: 1.11025071144104
Time to Process 6: 2.425602436065674


Token indices sequence length is longer than the specified maximum sequence length for this model (1253 > 512). Running this sequence through the model will result in indexing errors


Error processing row 7: The size of tensor a (1253) must match the size of tensor b (512) at non-singleton dimension 1
Time to Process 8: 44.624571561813354
Time to Process 9: 645.2929849624634
Time to Process 10: 0.0
Time to Process 11: 214.10998392105103
Time to Process 12: 61.27161169052124
Time to Process 13: 195.71793270111084
Time to Process 14: 44.30856275558472
Time to Process 15: 94.11770868301392
Time to Process 16: 165.51130437850952
Time to Process 17: 549.3984348773956
Error processing row 18: The size of tensor a (578) must match the size of tensor b (512) at non-singleton dimension 1
Time to Process 19: 48.47901654243469
Time to Process 20: 22.797882080078125
Time to Process 21: 0.0
Time to Process 22: 652.3311314582825
Time to Process 23: 2.637566566467285
Time to Process 24: 43.74749398231506
Time to Process 25: 20.486820697784424
Time to Process 26: 0.0
Time to Process 27: 0.0
Time to Process 28: 398.8521590232849
Time to Process 29: 10.726016998291016
Time to Process

In [25]:
data.to_csv("C:/Users/moazz/Box/Fintech Research Lab/Ethereum Governance Project/magician_comments_separated_withresults.csv")